# AC 221 - Problem Set 3

## Problem 1

### Finding Duplicates

In [3]:
import numpy as np
import csv
import datetime
import ipaddress
from copy import deepcopy

We begin by reading in the csv with the `csv` Python functionality and using a modified version of [Jim's `find_dups` function](https://github.com/jimwaldo/APCOMP221Code/blob/master/find_duplicates.py) to find duplicates.

#### Useful Functions

In [121]:
def get_key(x):
    return (x['user_id'], x['course_id'])
    
# Jim's find_dups Function (Modified to not print out csv with unique values)
"""
Created on 2019-03-14
Modified on 2020-04-02
@author waldo
"""
def find_dups(csv_in):
    total_lines = 0
    unique_lines = 0
    dup_lines = 0
    lines_seen = set()

    for l in csv_in:
        total_lines += 1
        key = get_key(l)
        if key not in lines_seen:
            lines_seen.add(key)
            unique_lines += 1
        else:
            dup_lines += 1
    return total_lines, unique_lines, dup_lines

def get_unique_users_courses(theRecords):
    total_records = 0
    unique_pairs = 0
    pairs_seen = set()

    for record in theRecords:
        total_records += 1
        key = get_key(record)
        if key not in pairs_seen:
            pairs_seen.add(key)
            unique_pairs += 1
    return total_records, unique_pairs, pairs_seen

def get_user(thePair):
    return thePair[0]

def get_missing_or_none_array(theRecord):
    missing_array = np.zeros((1, len(theRecord.keys())))
    for i, key in enumerate(theRecord.keys()):
        if theRecord[key] == '' or theRecord[key] == None:
            missing_array[0, i] = 1
    return missing_array

def get_sum_missing_pairs(theRecords, thePair):
    sums = []
    for record in grouped_records[thePair]:
        missings = get_missing_or_none_array(record)
        sums.append(np.sum(missings))
    return sums

# Based on Jim Waldo's find_dups function
"""
Created on 2019-03-14
@author waldo
"""
def create_unique_records_csv(csv_in, csv_out):
    total_lines = 0
    unique_lines = 0
    dup_lines = 0
    lines_seen = set()

    for l in csv_in:
        total_lines += 1
        key = get_key(l)
        if key not in lines_seen:
            lines_seen.add(key)
            csv_out.writerow(l)
            unique_lines += 1
        else:
            dup_lines += 1
    return total_lines, unique_lines, dup_lines

In [122]:
# Open original csv to get headers
with open("sample_set.csv", "r") as org_csv:
    reader = csv.reader(org_csv)
    headers = next(reader)

In [123]:
print(headers)

['course_id', 'user_id', 'registered', 'viewed', 'explored', 'certified', 'completed', 'ip', 'cc_by_ip', 'countryLabel', 'continent', 'city', 'region', 'subdivision', 'postalCode', 'un_major_region', 'un_economic_group', 'un_developing_nation', 'un_special_region', 'latitude', 'longitude', 'LoE', 'YoB', 'gender', 'grade', 'passing_grade', 'start_time', 'first_event', 'last_event', 'nevents', 'ndays_act', 'nplay_video', 'nchapters', 'nforum_posts', 'nforum_votes', 'nforum_endorsed', 'nforum_threads', 'nforum_comments', 'nforum_pinned', 'roles', 'nprogcheck', 'nproblem_check', 'nforum_events', 'mode', 'is_active', 'cert_created_date', 'cert_modified_date', 'cert_status', 'verified_enroll_time', 'verified_unenroll_time', 'profile_country', 'y1_anomalous', 'email_domain', 'language_brwsr', 'language_brwsr_country', 'language_brwsr_sec', 'language_brwsr_sec_country', 'language_brwsr_code', 'language_brwsr_subcode', 'language_brwsr_sec_code', 'language_brwsr_sec_subcode', 'language_brwsr_nev

#### Results for user_id/course_id Duplicates

In [5]:
# Run function
with open("sample_set.csv", "r", encoding = "utf8") as org_file:
    org_csv = csv.DictReader(org_file)        
    total_lines, unique_lines, dup_lines = find_dups(org_csv)

In [6]:
print("Total lines: {}".format(total_lines))
print("Unique lines: {}".format(unique_lines))
print("Duplicated lines: {}".format(dup_lines))

Total lines: 1100005
Unique lines: 137450
Duplicated lines: 962555


#### Investigating Nature of Duplicates

We want to know if it will be possible to use duplicates to fill in potential missing or bad values. We will see if these duplicates are exact record duplicates or only at the user_id/course_id level.

In [30]:
total_records, unique_pairs_n, unique_pairs = get_unique_users_courses(records)
print("Total records: {}".format(total_records))
print("Unique Pairs: {}".format(unique_pairs_n))

Total records: 1100005
Unique Pairs: 137450


In [31]:
sorted_pairs = sorted(unique_pairs, key = get_user)

In [32]:
sorted_records = sorted(records, key = get_key)

In [41]:
previous_key = ('', '')
full_duplicates = []

for i, record in enumerate(sorted_records):
    current_key = (record['user_id'], record['course_id'])
    if current_key == previous_key:
        if record == previous_record:
            full_duplicates.append(1)
        else:
            full_duplicates.append(0)
            print(current_key, previous_key)
    elif previous_key == ('', ''):
        previous_key = current_key
        previous_record = record  
    else:
        previous_record = record
        previous_key = current_key

In [42]:
if 0 in full_duplicates:
    print("Not all duplicates are complete record duplicates.")
else:
    print("All duplicates are complete record duplicates.")

All duplicates are complete record duplicates.


Looks like all the duplicates are the same! Thus, we can't use the duplicates to fill in potential missing oe bad values. We will have to repair values some other way.

For the rest of the problem set, we will be working with the file of unique records.

#### Creating Unique Record File

In [44]:
with open("sample_set_unique.csv", "w", encoding = "utf8") as unique_file:
    with open("sample_set.csv", "r", encoding = "utf8") as org_file:
        unique_csv = csv.DictWriter(unique_file, fieldnames = headers)
        org_csv = csv.DictReader(org_file)        
        total_lines, unique_lines, dup_lines = create_unique_records_csv(org_csv, unique_csv)

In [46]:
print("Total lines: {}".format(total_lines))
print("Unique lines: {}".format(unique_lines))
print("Duplicated lines: {}".format(dup_lines))

Total lines: 1100005
Unique lines: 137450
Duplicated lines: 962555


#### Checking with Pandas

We know from Problem 2 that there are no incomplete or malformed lines. That means when finding duplicates, we can use pandas and our only concern will be that the two fields we're interested in, course_id and user_id are read correctly. We can do that by enforcing their data types.

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('sample_set.csv', 
                 parse_dates=['cert_created_date','cert_modified_date', 
                              'start_time', 'first_event', 'last_event'],
                 dtype={
                     'course_id': np.str,
                     'user_id': np.int64,
                     'registered': np.bool, 
                     'explored': np.bool, 
                     'un_developing_nation': np.str},
                verbose=False)

ValueError: Bool column has NA values in column 4

In [ ]:
df.shape

In [ ]:
n_duplicates = df.duplicated(subset=['course_id', 'user_id'], keep='first').sum()
n_duplicates

There are 962,555 duplicate rows. This count includes all instances of each duplicated row except the first, so it is a count of how many rows would be removed if we were to remove duplicates. We can confirm this by actually dropping the duplicates and counting the rows.

In [ ]:
unique_rows = df.drop_duplicates(subset=['course_id', 'user_id'], keep='first')
unique_rows.shape

In [ ]:
n_duplicates + unique_rows.shape[0] == df.shape[0]

We get the same results as our more manual approach above.

### Repairing Values

By repairing values we mean to either:
* Replace bad values with correct values
* Impute missing values

First, we want to start by getting a better sense of our data to determine which values need to be repaired. This will also help us determine how to deal with missing values on a field by field basis. 

In [124]:
# Read and store unique records
records = []

with open("sample_set_unique.csv", "r", encoding = "utf8") as unique_file:
    unique_csv = csv.DictReader(unique_file, fieldnames = headers)
    for row in unique_csv:
        records.append(row)

In [126]:
len(records)

137450

#### Useful Functions

In [127]:
# Functions for exploring
def column_frequencies(theRecords, theColumn):
    frequency_dict = {}
    for record in theRecords:
        record_column = record[theColumn]
        if record_column in frequency_dict:
            frequency_dict[record_column] += 1
        else:
            frequency_dict[record_column] = 1
    return frequency_dict

def print_sorted_col_by_freq(frequencyDict):
    print({k: v for k, v in sorted(frequencyDict.items(), key=lambda item: item[1])})
    
def print_sorted_col_by_key(frequencyDict, n = 5):
    for i, key in enumerate(sorted(frequencyDict.keys())):
        if i < n:
            print(key, frequencyDict[key])
        
def print_frequencies_col_group(colGroup, n = 5):
    for col in colGroup:
        print(col)
        freqs = column_frequencies(records, col)
        print_sorted_col_by_key(freqs)
        print('')
        
def get_missing_frequency(theRecords, theColumn):
    freqs = column_frequencies(theRecords, theColumn)
    if '' in freqs.keys():
        return freqs['']
    else:
        return 0

def print_missing_frequencies(theRecords, colGroup):
    for col in colGroup:
        print("{}: {}".format(col, get_missing_frequency(theRecords, col)))

def cast_as_int(theRecord, theCol):
    try:
        theRecord[theCol] = int(theRecord[theCol])
    except ValueError:
        theRecord[theCol] = None
        
def cast_as_float(theRecord, theCol):
    try:
        theRecord[theCol] = float(theRecord[theCol])
    except ValueError:
        theRecord[theCol] = None
        
def remove_none_frequencies(theFrequencies):
    if None in theFrequencies.keys():
        del theFrequencies[None]
    return theFrequencies
        
def get_min(theRecords, theCol):
    freqs = column_frequencies(theRecords, theCol)
    freqs_no_none = remove_none_frequencies(freqs)
    return min(freqs_no_none.keys())

def get_max(theRecords, theCol):
    freqs = column_frequencies(theRecords, theCol)
    freqs_no_none = remove_none_frequencies(freqs)
    return max(freqs_no_none.keys())

def get_range(theRecords, theCol):
    return (get_min(theRecords, theCol), get_max(theRecords, theCol))

#### Find Bad or Missing Values

In order to determine what values are missing, we need to take a deeper look at the possible values in each column. We will also calculate the frequency of missing (the string '') for each column. We choose to not visualize the output as histograms and instead looked at the string output to make sure we were picking up any bad string values in expected numeric fields. For brevity, we will only output the frequencies of the first five values for each column, but we looked at all values.

In [164]:
# Take a look at columns by general categorical group
registration_cols = ['registered', 'viewed', 'explored', 'certified', 'completed', 'cert_status']
location_cols = ['cc_by_ip', 'countryLabel', 'continent', 'city', 'region', 'subdivision', 'postalCode', 
                 'profile_country', 'latitude', 'longitude']
un_cols = ['un_major_region', 'un_economic_group', 'un_developing_nation', 'un_special_region']
demo_cols = ['LoE', 'YoB', 'gender']
grade_cols = ['grade', 'passing_grade']
time_date_cols = ['start_time', 'first_event', 'last_event', 'cert_modified_date', 'cert_created_date', 'verified_enroll_time',
                  'verified_unenroll_time'] 
action_cols = ['nevents', 'ndays_act'] 
forum_cols = ['nforum_posts', 'nforum_votes', 'nforum_endorsed', 'nforum_threads', 'nforum_comments', 'nforum_pinned', 
              'roles', 'nprogcheck', 'nproblem_check', 'nforum_events'] 
language_cols = ['language_brwsr', 'language_brwsr_country', 'language_brwsr_sec', 'language_brwsr_sec_country', 
                 'language_brwsr_code', 'language_brwsr_subcode', 'language_brwsr_sec_code', 'language_brwsr_sec_subcode', 
                 'language_brwsr_nevents', 'language_brwsr_ndiff', 
                 'language', 'language_download', 'language_nevents', 'language_ndiff']
video_cols = ['ntranscript', 'nshow_answer', 'nvideo', 'nplay_video', 'nchapters', 'nvideos_unique_viewed', 'nvideos_total_watched', 
              'nseq_goto', 'nseek_video', 'npause_video']
dt_cols = ['avg_dt', 'sdv_dt', 'max_dt', 'n_dt', 'sum_dt']
roles_cols = ['roles_isBetaTester', 'roles_isInstructor', 'roles_isStaff', 'roles_isCCX', 'roles_isFinance', 'roles_isLibrary',
              'roles_isSales', 'forumRoles_isAdmin', 'forumRoles_isCommunityTA', 'forumRoles_isModerator', 
              'forumRoles_isStudent']
other_cols = ['mode', 'is_active', 'y1_anomalous', 'email_domain']

*Registration Fields*

In [59]:
for registration_col in registration_cols:
    print(registration_col)
    print(column_frequencies(records, registration_col))

registered
{'True': 137450}
viewed
{'True': 71210, 'False': 66240}
explored
{'False': 56393, 'True': 14817, '': 66240}
certified
{'False': 135038, 'True': 2412}
completed
{'False': 134515, 'True': 2935}
cert_status
{'': 72857, 'downloadable': 2412, 'notpassing': 60837, 'audit_notpassing': 1272, 'audit_passing': 40, 'unverified': 29, 'unavailable': 3}


In [23]:
print_missing_frequencies(records, registration_cols)

registered: 0
viewed: 0
explored: 531202
certified: 0
completed: 0
cert_status: 582030


In [62]:
explored_false_missing = 66240 + 14817
explored_false_missing

81057

Observations:
* Are there records where `certified` or `completed` are "True" but `explored` is "False" or Missing?
* We expected `explored` missing or False values might be related to missing values for other variables, particularly those with the same amount of missing such as the forum fields. However, this does not seem to be the case.  

*Demographic Fields*

In [63]:
print_frequencies_col_group(demo_cols)

LoE
 19982
a 6175
b 40552
el 608
hs 29262

YoB
 20873
1893 1
1894 6
1895 7
1896 4

gender
 18221
f 43041
m 75482
null 3
o 703



Observations
* `LoE` appears to be fine. We decided not to impute as we do not expect values to be missing completely at random. The distributon, at the very least, is probably dependent on the course level. (For example, smeone with a Ph.D probably isn't going to be taking an elementary school course.)
* **All `YoB` values, besides missing, are numeric. However, there are quite a few unbelievable values, particularly years in the 1800's, low 1900's, and 2010's. Thus, this variable is a candidate for repair.**
* **`gender` has both "null" and missing values which we expect to be equivalent in practice. Null might have some significance we don't understand, but considering its small frequency, we are going to treat it as missing. This variable is a candidate for repair.**
* We are generally erring on the side of caution when imputing variables. We expect `YoB`, like `LoE` to be correlated with the level of the course among other factors and so we will not impute 

In [64]:
print_frequencies_col_group(grade_cols)

grade
 43131
0.0 85934
0.01 570
0.02 354
0.03 386

passing_grade
 3488
0.5 8754
0.55 129
0.58 942
0.6 74692



In [65]:
print_missing_frequencies(records, grade_cols)

grade: 43131
passing_grade: 3488


Observations
* The `grade` variable mostly looks reasonable, but there are a few values above 1.00. We suspect some courses offered extra credit. Ideally, we would want to check such courses to make sure this is not an error.
* `passing_grade` looks reasonable. Documentation says it should be the same within a course. Ideally, we would check that each course does have the same value for this variable across all students.

*Date and Time Fields*

In [115]:
print_frequencies_col_group(time_date_cols)

start_time
2012-07-24 05:41:08 1
2012-07-24 09:27:16 1
2012-07-24 10:03:20 1
2012-07-24 11:39:48 1
2012-07-24 11:52:00 1

first_event
 31897
2012-09-07 01:17:18.096301 1
2012-09-07 06:34:28.682564 1
2012-09-07 08:03:47.346436 1
2012-09-07 09:23:22.404463 1

last_event
 31897
2012-09-07 06:35:12.352788 1
2012-09-07 09:49:37.456542 1
2012-09-08 16:03:45.943286 1
2012-09-08 18:59:09.373412 1

cert_modified_date
 72857
2013-02-01 16:00:58 1
2013-02-01 16:01:04 1
2013-02-01 16:01:08 1
2013-02-01 16:01:11 2

cert_created_date
 72857
2013-02-01 16:00:58 1
2013-02-01 16:01:03 1
2013-02-01 16:01:08 1
2013-02-01 16:01:11 2

verified_enroll_time
 135738
2014-08-07 20:31:17.314671 1
2014-09-13 14:59:12.815957 1
2014-09-13 21:25:43.286063 1
2014-10-08 13:07:14.611714 1

verified_unenroll_time
 137363
2014-10-30 15:54:32.908670 1
2014-12-02 22:25:02.047961 1
2014-12-30 17:20:32.154019 1
2014-12-30 22:46:29.573773 1



In [133]:
for record in records:
    for col in time_date_cols:
        if record[col] != '':
            try:
                datetime.datetime.strptime(record[col], '%Y-%m-%d %H:%M:%S.%f')
            except ValueError:
                try:
                    datetime.datetime.strptime(record[col], '%Y-%m-%d %H:%M:%S')
                except ValueError:
                    print(record[col])

In [159]:
print_missing_frequencies(records, time_date_cols)

start_time: 0
first_event: 31897
last_event: 31897
cert_modified_date: 72857
cert_created_date: 72857
verified_enroll_time: 135738
verified_unenroll_time: 137363


Observations:
* All non-missing date and time fields appear to be in the right format.

*Action Fields*

In [161]:
print_frequencies_col_group(action_cols)

nevents
 31903
1 15588
10 1531
100 125
1000 9

ndays_act
 31897
1 40082
10 1405
100 10
101 10



In [162]:
print_missing_frequencies(records, action_cols)

nevents: 31903
ndays_act: 31897


Observations
* These fields all look generally okay. All values are numeric. Registration for the course appears to count as an event and so the counts for these variables start at 1.

*Forum Fields*

In [68]:
print_frequencies_col_group(forum_cols)

nforum_posts
 133715
1 1508
10 54
104 1
105 1

nforum_votes
 133715
0 2601
1 481
10 15
11 15

nforum_endorsed
 133715
0 3681
1 39
14 1
2 5

nforum_threads
 133715
0 1620
1 1113
10 25
11 20

nforum_comments
 133715
0 1123
1 1065
10 39
100 2

nforum_pinned
 133715
0 3729
1 1
2 1
31 1

roles
Staff 130
Student 137320

nprogcheck
 31903
0 99229
1 2259
10 87
100 1

nproblem_check
 31903
0 93133
1 63
10 313
100 37

nforum_events
 31903
0 98661
1 1423
10 145
100 8



In [69]:
print_missing_frequencies(records, forum_cols)

nforum_posts: 133715
nforum_votes: 133715
nforum_endorsed: 133715
nforum_threads: 133715
nforum_comments: 133715
nforum_pinned: 133715
roles: 0
nprogcheck: 31903
nproblem_check: 31903
nforum_events: 31903


Observations
* These fields look okay. All values are numeric.
* A bit strange that `nforum_posts` starts at 1 when all other forum variables start at 0!

*Language Fields*

In [70]:
print_frequencies_col_group(language_cols)

language_brwsr
 84698
Afrikaans 1
Albanian 1
Arabic 263
Bengali 1

language_brwsr_country
 86205
Albania 1
Argentina 248
Australia 240
Austria 17

language_brwsr_sec
 121488
Afrikaans 21
Albanian 6
Amharic 1
Arabic 401

language_brwsr_sec_country
 129549
Afghanistan 1
Algeria 1
Argentina 8
Australia 20

language_brwsr_code
 84693
af 1
ar 263
bg 44
bn 1

language_brwsr_subcode
 84693
419 379
AE 18
AL 1
AR 248

language_brwsr_sec_code
 119405
 en 1982
 fr 55
 zh 2
af 21

language_brwsr_sec_subcode
 129219
419 158
AE 6
AF 1
AR 8

language_brwsr_nevents
 65425
1 4461
10 1117
100 114
1000 5

language_brwsr_ndiff
 65425
1 63726
2 6793
3 1196
4 238

language
 88835
bn 212
de 4
en 48096
es 147

language_download
 88835
0 47084
1 937
10 13
11 6

language_nevents
 88835
1 6734
10 945
100 67
1001 1

language_ndiff
 88835
1 46922
10 1
2 1194
3 316



In [71]:
print_missing_frequencies(records, language_cols)

language_brwsr: 84698
language_brwsr_country: 86205
language_brwsr_sec: 121488
language_brwsr_sec_country: 129549
language_brwsr_code: 84693
language_brwsr_subcode: 84693
language_brwsr_sec_code: 119405
language_brwsr_sec_subcode: 129219
language_brwsr_nevents: 65425
language_brwsr_ndiff: 65425
language: 88835
language_download: 88835
language_nevents: 88835
language_ndiff: 88835


Observations
* The language categorical fields look reasonable. 
* Some of the integer fields follow the same pattern as other integer fields where some fields start at 1 while others start at 0.

*Video Fields*

In [165]:
print_frequencies_col_group(video_cols)

ntranscript
 31903
0 90467
1 4477
10 226
100 9

nshow_answer
 31903
0 98319
1 1178
10 181
100 6

nvideo
 31903
0 60270
1 4536
10 869
100 76

nplay_video
 31903
0 60270
1 4536
10 869
100 76

nchapters
 66240
1 29739
10 608
11 618
12 296

nvideos_unique_viewed
 91124
1 12603
10 821
100 15
101 10

nvideos_total_watched
 91124
0.0022371364653243847 48
0.002242152466367713 3187
0.0025575447570332483 18
0.0026041666666666665 12

nseq_goto
 31903
0 74891
1 4133
10 689
100 26

nseek_video
 31903
0 76032
1 2584
10 742
100 36

npause_video
 31903
0 63626
1 5268
10 879
100 56



In [166]:
print_missing_frequencies(records, video_cols)

ntranscript: 31903
nshow_answer: 31903
nvideo: 31903
nplay_video: 31903
nchapters: 66240
nvideos_unique_viewed: 91124
nvideos_total_watched: 91124
nseq_goto: 31903
nseek_video: 31903
npause_video: 31903


Observations
* Some count variables start at 0 while others start after 0. This is directly related to the missing number differences across the video fields. It seems reasonable to assume that all these fields are being taken from the same pool of data: that is, if a record has a missing value for one of these fields, it has a missing value for all fields. The `nvideos_unique_viewed` and `nvideos_total_watched` have more missing data because they do not include 0. But because `nvideo` is a count of all video events, no matter the kind, we suspect that 60270 of the 91124 of the missing values of those two variables are actually 0. This matches the count of `nplay_video` with a value of 0.
* `nvideos_total_watched` is mostly from 0 to 1, but goes up to 6.5. This fields should capture the percentage of all videos watched in a course. We go more into detail in Problem 3.

Further Investigation

In [167]:
nvideo_flags = []

for record in records:
    if record['nvideo'] == '0' and record['nvideos_total_watched'] == '' and record['nvideos_unique_viewed'] == '':
        nvideo_flags.append(1)
    elif record['nvideo'] == '0' and (record['nvideos_total_watched'] != '' and record['nvideos_unique_viewed'] != ''):
        nvideo_flags.append(0)
    elif record['nvideo'] == '0' and (record['nvideos_total_watched'] != '' or record['nvideos_unique_viewed'] != ''):
        nvideo_flags.append(-1)
    elif record['nvideo'] == '' and (record['nvideos_total_watched'] == '' or record['nvideos_unique_viewed'] == ''):
        nvideo_flags.append(-2)
    elif record['nvideo'] == '' and (record['nvideos_total_watched'] != '' or record['nvideos_unique_viewed'] != ''):    
        nvideo_flags.append(-3)

In [168]:
print(len(nvideo_flags))
for j in [1, 0, -1, -2, -3]:
    print(sum(1 for i in nvideo_flags if i == j))

92173
59502
768
0
31517
386


In [169]:
i = 0

for record in records:
    if i < 10:
        if record['nvideo'] == '0' and (record['nvideos_total_watched'] != '' and record['nvideos_unique_viewed'] != ''):
            print(record['nvideos_total_watched'], record['nvideos_unique_viewed'])
            i += 1

0.02702702702702703 1
0.05405405405405406 2
0.05405405405405406 2
0.05405405405405406 2
0.02702702702702703 1
0.02702702702702703 1
0.05405405405405406 2
0.02702702702702703 1
0.023255813953488372 1
0.023255813953488372 1


Our suspicion was mostly accurate: 59502 of the 60270 records with 0 `nvideo` had missing values for the other two variables. There were 786 records that had 0 video events from the tracking logs but somehow had recorded a non-zero number of unique videos watched. We suspect that the number of videos watched is calculated separately from the tracking logs, or there is some sort of error.

*Time Difference Fields*

In [137]:
print_frequencies_col_group(dt_cols)

avg_dt
 54778
0.0 5
0.000274 1
0.000275 1
0.000277 1

sdv_dt
 61344
0.0 4
0.0007071067811864697 1
0.0007071067811865475 1
0.0033820917344152437 1

max_dt
 54778
0.0 5
0.000274 1
0.000275 1
0.000277 1

n_dt
 31897
0 22881
1 5732
10 1321
100 145

sum_dt
 54778
0.0 5
0.000274 1
0.000275 1
0.000277 1



In [105]:
print_missing_frequencies(records, dt_cols)

avg_dt: 54778
sdv_dt: 61344
max_dt: 54778
n_dt: 31897
sum_dt: 54778


Obserations
* None of the values look unreasonable and all values are numeric, but we are generally unaware what the bounds should be. This might vary by course. (For example, a course that is only open for $x$ total seconds should not have a `max_dt` value exceeding $x$.) When looking at ranges below, we will be able to see if the largest values seem to be at a reasonable scale.
* The difference in missing values across these five variables is unexpected as we expected these variables to be collected from the same source. (For example, why would a standard deviation of time difference exist but not the number of time difference events which are necessary for the computations?)

*Role Fields*

In [138]:
print_frequencies_col_group(roles_cols)

roles_isBetaTester
 137439
1 11

roles_isInstructor
 137386
1 64

roles_isStaff
 137392
1 58

roles_isCCX
 137450

roles_isFinance
 137449
1 1

roles_isLibrary
 137450

roles_isSales
 137449
1 1

forumRoles_isAdmin
 137434
1 16

forumRoles_isCommunityTA
 137450

forumRoles_isModerator
 137436
1 14

forumRoles_isStudent
 6
1 137444



Observations
* All fields are either 1 or missing. This suggests there is no "true" missing value as 0 (definitely not part of the role) and missing appear to have been collapsed together.
* No one in this sample has role of CCX, Library, or CommunityTA. If we knew more about the sample, this might be a potential redflag. 

*IP Address*

In [135]:
for record in records:
    if record['ip'] != '':
        try:
            ipaddress.ip_address(record['ip'])
        except ValueError:
            print(record['ip'])

In [139]:
get_missing_frequency(records, 'ip')

20698

IP Address Questions
* All non-missing IP Addresses had a correct format.

*Location Fields*

In [140]:
print_frequencies_col_group(location_cols)

cc_by_ip
 23167
AD 3
AE 430
AF 27
AG 9

countryLabel
 23167
Afghanistan 27
Albania 113
Algeria 201
American Samoa 1

continent
 22970
Africa 6219
Americas 48453
Asia 28019
Europe 24161

city
 38883
's-gravendeel 1
's-hertogenbosch 4
't Horntje 1
A Coruña 17

region
 43383
00 403
01 634
02 242
025 1

subdivision
 41248
A Coruña 40
Aargau 13
Aberdeen City 18
Aberdeenshire 1

postalCode
 84682
00011 1
00012 1
0002 1
00027 1

profile_country
 52674
AD 7
AE 286
AF 79
AG 7

latitude
 22970
-0.0333 1
-0.1 1
-0.2167 174
-0.25 1

longitude
 22970
-0.0001 6
-0.0013 5
-0.0031 4
-0.0104 1



In [141]:
print_missing_frequencies(records, location_cols)

cc_by_ip: 23167
countryLabel: 23167
continent: 22970
city: 38883
region: 43383
subdivision: 41248
postalCode: 84682
profile_country: 52674
latitude: 22970
longitude: 22970


Observations
* The first three city names that begin with an apostrophe might have been an encoding error, but no, those are the real names of the cities in the Netherlands.
* Below we will check that latitude and longitude are in the proper range.

In [142]:
print_frequencies_col_group(un_cols)

un_major_region
 23167
Australia and New Zealand 2698
Caribbean 724
Central America 2679
Central Asia 220

un_economic_group
 23167
Developed regions 69462
Developing_Nations 44821

un_developing_nation
 135362
Least developed countries 2088

un_special_region
 121866
Latin America and the Caribbean 11460
Sub-Saharan-Africa 4124



In [143]:
print_missing_frequencies(records, un_cols)

un_major_region: 23167
un_economic_group: 23167
un_developing_nation: 135362
un_special_region: 121866


Observations:
* As with the roles fields, there is no distinction between legitimate missing and not part of any of the categories. We could use `un_major_region` to get a "real" missing category and "Not a Developing Nation" or "Not a Special Region" categories. 
* **n_developing_nation` and `un_special_region` are candidates for repair.**

*Misc Fields*

In [144]:
print_frequencies_col_group(other_cols)

mode
 917
audit 60266
honor 74474
verified 1793

is_active
 917
0 20482
1 116051

y1_anomalous
 136531
1 919

email_domain
013.net 1
10count.ca 1
126.COM 1
126.com 232
139.com 9



In [145]:
print_missing_frequencies(records, other_cols)

mode: 917
is_active: 917
y1_anomalous: 136531
email_domain: 0


Observations
* We expected `y1_anomalous` might be tied to some missing data since the user "had to be side loaded from mongo" but this variable seems to only be obviously related to `mode` and `is_active`. 
* **Email domain could use some cleaning - forcing everything to lower case. This is a candidate for repair.**

*Numeric Fields*

We did not find any bad string values in expected numeric fields. Thus, we can cast those fields as float or integer for easier analysis of the fields.

In [146]:
def cast_as_int(theRecord, theCol):
    try:
        theRecord[theCol] = int(theRecord[theCol])
    except ValueError:
        theRecord[theCol] = None
        
def cast_as_float(theRecord, theCol):
    try:
        theRecord[theCol] = float(theRecord[theCol])
    except ValueError:
        theRecord[theCol] = None
        
def remove_none_frequencies(theFrequencies):
    if None in theFrequencies.keys():
        del theFrequencies[None]
    return theFrequencies
        
def get_min(theRecords, theCol):
    freqs = column_frequencies(theRecords, theCol)
    freqs_no_none = remove_none_frequencies(freqs)
    return min(freqs_no_none.keys())

def get_max(theRecords, theCol):
    freqs = column_frequencies(theRecords, theCol)
    freqs_no_none = remove_none_frequencies(freqs)
    return max(freqs_no_none.keys())

def get_range(theRecords, theCol):
    return (get_min(theRecords, theCol), get_max(theRecords, theCol))

In [113]:
# Cast appropriate fields as numeric
records_with_numeric = deepcopy(records)

for record in records_with_numeric:
    cast_as_float(record, 'latitude')
    cast_as_float(record, 'longitude')
    cast_as_int(record, 'YoB')
    cast_as_float(record, 'grade')
    cast_as_float(record, 'passing_grade')
    cast_as_int(record, 'nevents')
    cast_as_int(record, 'ndays_act')
    cast_as_int(record, 'nplay_video')
    cast_as_int(record, 'nchapters')
    cast_as_int(record, 'nforum_posts')
    cast_as_int(record, 'nforum_votes')
    cast_as_int(record, 'nforum_endorsed')
    cast_as_int(record, 'nforum_threads')
    cast_as_int(record, 'nforum_comments')
    cast_as_int(record, 'nforum_pinned')
    cast_as_int(record, 'nprogcheck')
    cast_as_int(record, 'nproblem_check')
    cast_as_int(record, 'nforum_events')
    cast_as_int(record, 'language_brwsr_nevents')
    cast_as_int(record, 'language_brwsr_ndiff')
    cast_as_int(record, 'language_download')
    cast_as_int(record, 'language_nevents')
    cast_as_int(record, 'language_ndiff')
    cast_as_int(record, 'ntranscript')
    cast_as_int(record, 'nshow_answer')
    cast_as_int(record, 'nvideo')
    cast_as_int(record, 'nvideos_unique_viewed')
    cast_as_float(record, 'nvideos_total_watched')
    cast_as_int(record, 'nseq_goto')
    cast_as_int(record, 'nseek_video')
    cast_as_int(record, 'npause_video')
    cast_as_float(record, 'avg_dt')
    cast_as_float(record, 'sdv_dt')
    cast_as_float(record, 'max_dt')
    cast_as_float(record, 'n_dt')
    cast_as_float(record, 'sum_dt')

Look at Ranges of Numeric Values

In [147]:
quant_cols = ['latitude', 'longitude', 'YoB', 'grade', 'passing_grade', 
              'nevents', 'ndays_act', 'nplay_video', 'nchapters', 'nforum_posts', 'nforum_votes', 'nforum_endorsed', 
              'nforum_threads', 'nforum_comments', 'nforum_pinned', 'nprogcheck', 'nproblem_check', 'nforum_events', 
              'language_brwsr_nevents', 'language_brwsr_ndiff', 'language_nevents', 'language_ndiff', 'ntranscript', 
              'nshow_answer', 'nvideo', 'nvideos_unique_viewed', 'nvideos_total_watched', 'nseq_goto', 'nseek_video', 
              'npause_video', 'avg_dt', 'sdv_dt', 'max_dt', 'n_dt', 'sum_dt']

for col in quant_cols:
    print("{}: {}".format(col, get_range(records_with_numeric, col)))

latitude: (-54.8, 71.285)
longitude: (-175.2018, 178.4167)
YoB: (1893, 2018)
grade: (0.0, 1.15)
passing_grade: (0.5, 0.97)
nevents: (1, 234385)
ndays_act: (1, 2475)
nplay_video: (0, 90057)
nchapters: (1, 34)
nforum_posts: (1, 299)
nforum_votes: (0, 224)
nforum_endorsed: (0, 30)
nforum_threads: (0, 142)
nforum_comments: (0, 281)
nforum_pinned: (0, 31)
nprogcheck: (0, 643)
nproblem_check: (0, 26367)
nforum_events: (0, 17829)
language_brwsr_nevents: (1, 114745)
language_brwsr_ndiff: (1, 8)
language_nevents: (1, 6051)
language_ndiff: (1, 10)
ntranscript: (0, 4346)
nshow_answer: (0, 3432)
nvideo: (0, 90057)
nvideos_unique_viewed: (1, 363)
nvideos_total_watched: (0.0022371364653243847, 6.5)
nseq_goto: (0, 6941)
nseek_video: (0, 10398)
npause_video: (0, 57069)
avg_dt: (0.0, 299.589759)
sdv_dt: (0.0, 209.66384207378675)
max_dt: (0.0, 300.0)
n_dt: (0.0, 233917.0)
sum_dt: (0.0, 2895426.074810009)


Observations
* Latitude and longitude are in the right ranges.
* As we mentioned above, some count variables start at 1 while others at 0. 
    * For `nevents` and `ndays_act`, a minimum of 1 is reasonable if registering for the course counts as an activity. `nchapters` might be similar if the user automatically sees the first chapter upon registering. 
    * For the language browser fields, a minimum value of 1 is also reasonable if all studenst that do not engage with the video transcript receive a missing value.
    * For other variables that have both, what's the difference between 0 and missing? We decided that some more digging would have to be done before repairing such values. Our final dataset would include documentation explaining that some count variables do not include 0. 
* The total elapsed time spent on a course `sum_dt` was around 33 days, which seems reasonable.

#### Repairing Values

We will repair the dataset with duplicates removed and with fields still as strings. We erred on the side of caution in repairing values because we do not know what the use of the dataset would be. We only repaired values for which we expected little or no bias introduced or were confident that we knew what the true value should be. For this reason, we would also leave imputation up to the individual analysts and would instead focus on providing strong documentation. 

In [148]:
# Create deepcopy of records
cleaned_records = deepcopy(records)

*YoB*

This variable included a few unbelievable values, particularly years in the 1800's, low 1900's, and 2010's. We will use  somewhat arbitrary (and slightly ageist) cutoffs. We collapsed unreasonable values into string categories. Our cutoffs were:
* Any value below 1920 (which would put the student at 92-years-old if the course was at the time of edX's launch)
* Any value above 2007 (which would put the student at about 13 years-old if the course was from this year)

A better, but much more involved, method would be to consider the year in which the course ran and try to correct for values where a student incorrectly put their birth year as the current year. 

In [156]:
for record in cleaned_records:
    if record['YoB'] != '':
        if int(record['YoB']) < 1920:
            record['YoB'] = "< 1920"
        elif int(record['YoB']) > 2007:
            record['YoB'] = "> 2007"

*Gender*

This field had 3 'null' values that we will collapse as missing. Null might have some significance that we don't understand, but we feel confident collapsing the value into missing because of its small frequency.

In [149]:
for record in cleaned_records:
    if record['gender'] == 'null':
        record['gender'] = ''

In [150]:
column_frequencies(cleaned_records, 'gender')

{'': 18224, 'm': 75482, 'f': 43041, 'o': 703}

*Email Domain*

We want  the email domain variable to hold completely lower-case strings for proper grouping.

In [151]:
for record in cleaned_records:
    record['email_domain'] = record['email_domain'].lower()

*UN Fields*

We assumed that if the `un_major_region` field was not missing, it was possible to determine if the region was a developing nation or special region. Thus, we created a category to capture regions that were not really missing but did not fit in the current categories. This resulted in an equal missing rate across all four variables.

In [154]:
for record in cleaned_records:
    if record['un_developing_nation'] == '':
        if record['un_major_region'] != '':
            record['un_developing_nation'] = "Not a least developed country"
    if record['un_special_region'] == '':
        if record['un_major_region'] != '':
            record['un_special_region'] = "Not a UN special region"

In [155]:
print_missing_frequencies(cleaned_records, un_cols)

un_major_region: 23167
un_economic_group: 23167
un_developing_nation: 23167
un_special_region: 23167


*Video Fields*

We will repair the 59502 records with a value of 0 in `nvideo` but missing values for `nvideos_total_watched` and `nvideos_unique_viewed`. This leaves 1154 records with either 0 or missing `nvideo` values but nonzero, nonmissing values for the other two variables.

In [170]:
for record in cleaned_records:
    if record['nvideo'] == '0' and record['nvideos_total_watched'] == '' and record['nvideos_unique_viewed'] == '':
        record['nvideos_total_watched'] = 0
        record['nvideos_unique_viewed'] = 0

### Cleaned Dataset

In [174]:
with open("sample_set_unique_cleaned.csv", "w", encoding = "utf8") as cleaned_file:
    cleaned_csv = csv.DictWriter(cleaned_file, fieldnames = headers)
    cleaned_csv.writeheader()
    cleaned_csv.writerows(cleaned_records)

## Problem 2

There are numerous ways a line could be corrupt. We will consider lines corrupt that either fail to parse as CSV lines or have fewer fields than the header specifies. First, let's see how many newline characters are in the file. This is an upper bound on the number of rows our CSV should have (because fields can contain newline characters).

In [7]:
!wc -l sample_set.csv

'wc' is not recognized as an internal or external command,
operable program or batch file.


There are 1,100,006 lines in the CSV. We can see below that the first line is a header and the second line is where data begins.

In [ ]:
!head -n 2 sample_set.csv


So we'd expect at most 1,100,005 rows of data.

We already have some indication from Problem 1 that there are exactly this number of rows if pandas parses the file for us. Let's do some double checks. For our second pass at counting the number of good rows, we will use an adapted version of the code Professor Waldo showed in class.

In [ ]:
import csv, sys

def count_line_status(csv_in):
    total_lines = 0
    good_lines = 0
    bad_lines = 0

    header = next(csv_in)
    l_len = len(header)
    while True:
        try:
            total_lines += 1
            l = next(csv_in)
            if len(l) == l_len:
                good_lines += 1
            else:
                bad_lines += 1
        except StopIteration:
            total_lines -= 1
            break
        except:
            bad_lines += 1
            continue
    return total_lines, good_lines, bad_lines

In [ ]:
with open('sample_set.csv') as f:
    reader = csv.reader(f)
    total_lines, good_lines, bad_lines = count_line_status(reader)
print('Total lines = ' + str(total_lines), 'Good lines = ' + str(good_lines),
          'Bad lines = ' + str(bad_lines))

This gives us further evidence there are zero corrupt lines. Every row parses correctly and every row has exactly 91 fields. Let's do that same thing one more time, but with the CSV parser set to strict so we know it will throw exceptions.

In [ ]:
class MyDialect(csv.Dialect):
    strict = True
    skipinitialspace = True
    quoting = csv.QUOTE_ALL
    delimiter = ','
    quotechar = '"'
    lineterminator = '\n'
    
with open('sample_set.csv') as f:
    reader = csv.reader(f, MyDialect())
    total_lines, good_lines, bad_lines = count_line_status(reader)
    
print('Total lines = ' + str(total_lines), 'Good lines = ' + str(good_lines),
          'Bad lines = ' + str(bad_lines))

We're told again there are zero bad lines. As one final check, let's count the number of commas in each line. A well-structured line should have 90 commas separating the 91 fields. If a line was truncated, it will likely have fewer than 90. This is not a strict test because lines may have commas that are not field delimiters but instead part of the field value, but it will likely provide some indication.

In [ ]:
import collections
n_commas_per_line = []
with open('sample_set.csv') as f:
    for line in f:
        n_commas = len(line.split(','))-1
        n_commas_per_line.append(n_commas)
collections.Counter(n_commas_per_line)

We see that every line has at least 90 commas, so between this result and those above, it's quite likely there are no truncated or unparsable lines. As this make answering the rest of the problem less meaningful, I asked on Piazza and was given another dataset which assuredly has corrupt lines against which to test the above methods. Let's see if they work.

In [ ]:
with open('sample_set2.csv') as f:
    reader = csv.reader(f)
    total_lines, good_lines, bad_lines = count_line_status(reader)
print('Total lines = ' + str(total_lines), 'Good lines = ' + str(good_lines),
          'Bad lines = ' + str(bad_lines))

Bad lines are detected here, so it's safe to assume our method works and there are no bad lines in the original file.

How many corrupt lines are there? Zero.

Does the count of corrupt lines change if you get rid of them before getting rid of the duplicate records? No. It's still zero.

What difference might this make to the remaining data set? None. There are zero corrupt lines.

## Problem 3

### Sources of Bias
* We don't really know how the dataset was collected, so there could potentially be bias concerns in collection.
* We also don't know how this dataset was sampled. The name of the file suggests there was some sampling that happened somewhere during the construction of this dataset, and it's possible bias was introduced by the sampling procedure.
* We don't know why there are duplicate lines.

### Documentation of Unusual Records or Values

#### General
* There are a large number of duplicate lines.
* There are no corrupt lines. We had some expectation that there would be, and given that there aren't, we may want to investigate why there aren't corrupt lines.

#### Year of Birth
* The original file included some unusual values for year of birth in the 1800's, low 1900's, and 2010's. We created categorical flags for these values. We suggest that analysts using our cleaned dataset create a separate numeric column that sets such values as missing.

#### Count Variables
We urge analysts using this dataset pay special attention to count variables as they sometimes do not include 0.
* `nevents` and `ndays_act` do not include 0 because registration counts as an event.
* We are not sure why `nforum_posts` does not include 0.
* We are not sure why `nchapters` does not include 0.
* For the language browser fields, a minimum value of 1 is also reasonable if all students that do not engage with the video transcript receive a missing value.
* nvideos_total_watched is mostly from 0 to 1, but goes up to 6.5. This fields should capture the percentage of all videos watched in a course. Possible reasons for this anomaly are:
    * We suspect if the value is greater than 1, a student is watching videos multiple times. (This might mean that a student has not actually watched all videos, but watched some multiple times and others not at all. This would have quite an effect on analysis so we should try to figure out how this field is actually calculated.
    * If the value is greater than 1, it might be an issue in the calculation not scaling to percent. (For example, 6.5 should be 0.65 or 0.065.) There is no way of knowing this unless we know how this field is actually calculated.
    
#### Time Difference
* The difference in missing values across the five time difference variables is unusual as we expected these variables to be collected from the same source. (For example, why would a standard deviation of time difference exist but not the number of time difference events which are necessary for the computation of standard deviation?)

#### Unusual Binary Fields
* The role and forum role variables do not have values of 0, only 1 or missing. This means an analyst might not be able to differentiate between students who were truly not considered as that particular role versus for whom this data might be missing.
* Additionally, no one in the sample had a role of CCX, Library, or Community TA. If we knew more about the sample, this might be a potential redflag.